# Event Sanity Checks

This notebook contains sanity checks and exploratory queries used to validate the randomly generated financial event data for the LedgerOne project. The goal is to ensure data correctness and consistency before transforming events into ledger entries.

In [65]:
# Call 1: Setup

from pathlib import Path
import duckdb
import pandas as pd

PROJECT_ROOT = Path("/Users/trustanprice/Desktop/Personal/ledgerone")
DB_PATH = PROJECT_ROOT / "data/processed/ledgerone.duckdb"
SQL_PATH = PROJECT_ROOT / "sql/ledger.sql"

DATA_DIR = Path("/Users/trustanprice/Desktop/Personal/ledgerone/data/raw/")

EVENTS_PATH = DATA_DIR / "events.parquet"
USERS_PATH = DATA_DIR / "users.parquet"
ACCOUNTS_PATH = DATA_DIR / "accounts.parquet"

con = duckdb.connect()

In [66]:
# Cell 2: Load & inspect events
con.execute(f"""
  SELECT event_type, direction, COUNT(*) AS cnt
  FROM read_parquet('{EVENTS_PATH}')
  GROUP BY 1, 2
  ORDER BY 1, 2
""").df()

,event_type,direction,cnt
0,DEPOSIT,CREDIT,100
1,FEE,DEBIT,313
2,PURCHASE,DEBIT,313
3,REFUND,CREDIT,30


In [67]:
# Cell 3: Spot-check refunds
con.execute(f"""
  SELECT *
  FROM read_parquet('{EVENTS_PATH}')
  WHERE event_type = 'REFUND'
  LIMIT 10
""").df()


,event_id,event_ts,user_id,account_id,event_type,direction,amount,currency,reference_id
0,db0197f1-8652-4d8f-b439-03875f1dc6d2,2024-01-24 05:28:22,2dd3e3ec-19e1-47c3-bc16-edfb88d581cc,ebc4435c-93c8-4e35-929a-98d6ebf4815e,REFUND,CREDIT,10.0,USD,46dee498-96a2-465a-b13c-9df8a9227efb
1,d7af6c1d-caac-408c-8002-589e3420dd24,2024-03-29 14:50:39,302830ec-b9f9-4690-bf57-973e19bb0612,078c8ece-7c7b-4116-874b-4d95a251cea6,REFUND,CREDIT,25.0,USD,b4d97f73-33ec-4e79-abbc-2208f490d55d
2,ba04e06a-8615-4895-919a-aec2dc4a3219,2024-03-27 12:33:44,1cfae09f-e07d-4882-a5ba-f1da69d933d4,7fe40eaa-0ff4-42ab-ba1e-2ec2c5b7e7ae,REFUND,CREDIT,20.0,USD,9ea9bed0-c324-4d98-ab94-2fda2378f64b
3,5b5508f9-8692-4e21-80d3-f569c094ae0a,2024-01-06 06:43:46,157d4b75-8f20-458a-be03-ecd22329b7e2,fc3110b5-da6d-4763-ac37-e16886ffad6d,REFUND,CREDIT,20.0,USD,d7d72809-1185-45e2-8e9d-3c63ad618e18
4,c2ecd1d9-ba04-413a-a81c-d0867de0ec5f,2024-02-25 11:00:27,07e66529-b54c-418f-b110-f2be2b082e32,94c140c3-5ca1-42c7-b7c1-77f65fad360a,REFUND,CREDIT,25.0,USD,c0256516-0ab5-4ee7-bf85-761488d943d0
5,a5a1b9e1-6f66-499e-a80e-33d071a39e2f,2024-02-23 23:19:03,07e66529-b54c-418f-b110-f2be2b082e32,94c140c3-5ca1-42c7-b7c1-77f65fad360a,REFUND,CREDIT,15.0,USD,4526352a-cc7f-4a4e-871b-0944e7d904cd
6,6833bc46-cd8f-42a5-891c-373d2d046613,2024-01-23 15:26:58,0c75b59f-2b98-4504-9e36-20d710a24cc6,0fbb1b26-a61e-4389-b997-1a21be2a0570,REFUND,CREDIT,15.0,USD,8b4b4477-bfc2-4f73-96d0-1f814f25e271
7,cd300eed-3e95-40fa-b26e-a22ba633e1eb,2024-03-25 22:05:18,b057f2e3-8f21-47a1-b38a-81183cc3878b,49610a3c-2356-4283-8ac7-1d09b238f4ae,REFUND,CREDIT,20.0,USD,fffdf1fd-a04f-4466-bf22-df200f4c1583
8,69dc5777-2264-4c4f-8b9b-8fbae8c5ae7f,2024-01-26 17:32:47,290af576-8447-412a-8d38-2e7a8b629699,611cb3d7-e275-41ac-a90b-75a5fee4614b,REFUND,CREDIT,15.0,USD,0e055c77-f8b1-46f4-bc2a-32a643d1bdb9
9,65e6ac9b-2b8c-44ff-b2f3-70952d58e1c1,2024-03-05 23:14:00,6892f989-32e4-434d-92b8-525823422d7d,756bc02f-7d40-4288-8265-ef7961c36ce9,REFUND,CREDIT,15.0,USD,0fd2691c-8fd3-4f06-9311-2b6263996e54


In [68]:
# Cell 4: Check amounts are always positive
con.execute(f"""
  SELECT COUNT(*) AS negative_amounts
  FROM read_parquet('{EVENTS_PATH}')
  WHERE amount <= 0
""").df()

,negative_amounts
0,0


In [69]:
con = duckdb.connect(str(DB_PATH))

with open(SQL_PATH, "r") as f:
    con.execute(f.read())

rows = con.execute("SELECT COUNT(*) FROM ledger_entries").fetchone()[0]
print("ledger_entries rows:", rows)

con.close()

ledger_entries rows: 756
